In [1]:
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd
import psycopg2
from psycopg2 import sql
import plotly.express as px
import matplotlib.pyplot as plt
import folium
import os

# move current directory to /Users/tanyatsui/Documents/01_Projects/housingEmissions
os.chdir('/Users/tanyatsui/Documents/01_Projects/housingEmissions')

In [2]:
# create a connection to the database
db_name = 'urbanmining'
db_user = 'postgres'
db_password = 'Tunacompany5694!'
db_host = 'localhost'
db_port = '5432'
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


# Problem: Percentage change in embodied emissions 
~40% of municipalities have the exact same result for % change in embodied emissions for the energy efficiency scenario - 58.730159%. This can be seen in the perfectly straight horizontal line of dots in the scatterplot. 

In [30]:
query = f''' 
WITH emissions_with_municipality AS (
    SELECT b.municipality, a.wk_code, a.year, a.n_units, 
		a.embodied_kg_s0, a.embodied_kg_s1, 
		a.operational_kg_s0, a.operational_kg_s1
	FROM emissions_all_wijk_s1 a 
	LEFT JOIN key_wijk2012_to_municipality2022 b 
	ON a.wk_code = b.wk_code
), 
emissions_municipality AS (
    SELECT municipality, 
        SUM(operational_kg_s0) AS operational_kg_s0, SUM(operational_kg_s1) AS operational_kg_s1, 
        SUM(embodied_kg_s0) AS embodied_kg_s0, SUM(embodied_kg_s1) AS embodied_kg_s1, 
        SUM(operational_kg_s0 + embodied_kg_s0) AS total_kg_s0, SUM(operational_kg_s1 + embodied_kg_s1) AS total_kg_s1
    FROM emissions_with_municipality
    GROUP BY municipality
), 
emissions_stats AS (
    SELECT municipality, 
        operational_kg_s0, operational_kg_s1, embodied_kg_s0, embodied_kg_s1, total_kg_s0, total_kg_s1,
        operational_kg_s1 - operational_kg_s0 AS diff_operational,
        (operational_kg_s1 - operational_kg_s0) / operational_kg_s0 * 100 AS diff_operational_pct,
        embodied_kg_s1 - embodied_kg_s0 AS diff_embodied,
        (embodied_kg_s1 - embodied_kg_s0) / embodied_kg_s0 * 100 AS diff_embodied_pct,
        total_kg_s1 - total_kg_s0 AS diff_total,
        (total_kg_s1 - total_kg_s0) / total_kg_s0 * 100 AS diff_total_pct
    FROM emissions_municipality 
), 
municipalities AS (
    SELECT "GM_NAAM", geometry FROM nl_gemeenten WHERE "H2O" = 'NEE'
)
SELECT a.*, b.geometry
FROM emissions_stats a
LEFT JOIN municipalities b
ON a.municipality = b."GM_NAAM"
ORDER BY a.diff_total DESC
'''

gdf = gpd.read_postgis(query, engine, geom_col='geometry')

In [31]:
df = gdf[['municipality', 'embodied_kg_s0', 'embodied_kg_s1', 'diff_embodied_pct']].copy()
df['diff_embodied_pct_python'] = (df.embodied_kg_s1 - df.embodied_kg_s0) / df.embodied_kg_s0 * 100
df = df[(df.diff_embodied_pct >= 58.7) & (df.diff_embodied_pct <= 58.8)]
problem_municipalities = df.municipality.tolist()

print(f'{len(df) / len(gdf) * 100:.0f}% of the municipalities have the exact same result: 58.730159% increase in embodied emissions')
df.head()

42% of the municipalities have the exact same result: 58.730159% increase in embodied emissions


,municipality,embodied_kg_s0,embodied_kg_s1,diff_embodied_pct,diff_embodied_pct_python
0,Venlo,2.321153e+09,3.684370e+09,58.730159,58.730159
11,Almere,9.824409e+07,1.559430e+08,58.730159,58.730159
19,Olst-Wijhe,1.370981e+08,2.176160e+08,58.730159,58.730159
144,Nieuwegein,3.955921e+07,6.279240e+07,58.730159,58.730159
148,Emmen,5.480420e+07,8.699080e+07,58.730159,58.730159


In [4]:
fig = px.scatter(gdf, x='diff_operational_pct', y='diff_embodied_pct', 
                 hover_name='municipality', 
                 title="Percentage change in operational vs embodied emissions for Dutch municipalities")
fig.update_layout(width=800, height=700)
fig.show()

# Source of the problem: missing construction and demolition data 
For some reason, many municipalities are missing construction and demolition data. As a result, all activities in these municipalities are transformations and renovations. This leads to the same proportional change in emissions from s0 to s1, from 126/sqm to 200/sqm. This leads to exactly a 58.730158730158735% increase in emissions no matter how many sqm of renovations / transformations there are in the municipality. 

The problem comes from the creation of `housing_nl` in the `ConstructionActivityInfoAdder` module. It seems like the last step, `add_construction_and_demolition.sql`, didn't run properly for many of the municipalities. 

In [26]:
municipality = 'Venlo'
gdf_all = gpd.GeoDataFrame()
for year in range(2012, 2021 + 1): 
    query = f''' 
    -- get wijk_stats: wijk level energy use and in-use sqm data 
    WITH cbs_stats_wijk AS (
        SELECT * FROM cbs_map_all_wijk WHERE municipality = '{municipality}' AND year = {year}
    ), 
    housing_inuse AS (
        SELECT * FROM housing_inuse_2012_2021 WHERE municipality = '{municipality}' AND year = {year}
    ), 
    housing_inuse_wijk AS (
        SELECT municipality, wk_code, year, SUM(sqm) AS sqm, SUM(n_units) AS n_units
        FROM housing_inuse
        GROUP BY municipality, wk_code, year
    ), 
    wijk_stats AS (
        SELECT b.*, a.sqm, a.n_units 
        FROM housing_inuse_wijk a 
        JOIN cbs_stats_wijk b 
        ON a.municipality = b.municipality 
            AND a.year = b.year
            AND a.wk_code = b.wk_code 
    ), 

    -- get all constructions and renovations that happened before year  
    construction_municipality AS ( -- all construction activity (except for demolition) in year
        SELECT id_pand, 
            CASE 
                WHEN status = 'Pand gesloopt' THEN LEFT(registration_start, 4)::INTEGER
                WHEN status != 'Pand gesloopt' AND registration_end IS NOT NULL THEN LEFT(registration_end, 4)::INTEGER
                ELSE LEFT(registration_start, 4)::INTEGER
            END AS year, 
            status, sqm, n_units, geom, geom_28992, wk_code, municipality
        FROM housing_nl
        WHERE municipality = '{municipality}'
            AND ahn_version IS NULL
    ), 
    construction_sample AS (
        SELECT * FROM construction_municipality 
        WHERE year <= {year}
    ), 

    -- identify in-use buildings that were previously constructed or renovated (low-energy)
    inuse_lowenergy AS (
        SELECT 
            b.id_pand, b.year, 'Pand in gebruik - low energy' AS status, b.sqm, b.n_units, b.wk_code
        FROM (SELECT DISTINCT ON (id_pand) id_pand, status FROM construction_sample) a 
        LEFT JOIN housing_inuse b 
        ON a.id_pand = b.id_pand
        WHERE a.status != 'Pand gesloopt'
            AND b.id_pand IS NOT NULL
    ), 
    inuse_normalenergy AS (
        SELECT b.id_pand, b.year, 'Pand in gebruik' AS status, b.sqm, b.n_units, b.wk_code
        FROM construction_sample a 
        RIGHT JOIN housing_inuse b 
        ON a.id_pand = b.id_pand
        WHERE a.id_pand IS NULL 
    ), 
    buildings_all AS (
        -- all construction / renovation / transformation / demolition activity in year
        SELECT id_pand, year, status, sqm, n_units, wk_code 
        FROM construction_sample 
        WHERE year = {year}
        
        UNION ALL 
        
        -- low energy in use buildings in year
        SELECT * FROM inuse_lowenergy
        
        UNION ALL 
        
        -- non-low energy in use buildings in year
        SELECT * FROM inuse_normalenergy
    ), 

    -- calculate energy use per building according to low or normal energy use status
    energy_use_per_building AS (
        SELECT a.id_pand, a.year, a.status, a.sqm, a.n_units, 
            CASE 
                WHEN status IN ('Pand in gebruik', 'Pand in gebruik - low energy') THEN ROUND(a.sqm / b.sqm * b.gas_m3)
                ELSE 0
            END AS gas_m3_s0,
            CASE
                WHEN status = 'Pand in gebruik' THEN ROUND(a.sqm / b.sqm * b.gas_m3)
                WHEN status = 'Pand in gebruik - low energy' THEN a.sqm * 5
                ELSE 0 
            END AS gas_m3_s1,
            CASE 
                WHEN status IN ('Pand in gebruik', 'Pand in gebruik - low energy') THEN ROUND(a.sqm / b.sqm * b.elec_kwh) 
                ELSE 0 
            END AS electricity_kwh_s0, 
            CASE 
                WHEN status IN ('Pand in gebruik', 'Pand in gebruik - low energy') THEN ROUND(a.sqm / b.sqm * b.elec_kwh) 
                ELSE 0 
            END AS electricity_kwh_s1, 
            b.wk_code, b.wk_geom
        FROM buildings_all a 
        JOIN wijk_stats b 
        ON a.wk_code = b.wk_code
    ), 
    emissions_per_building AS (
        SELECT id_pand, year, status, sqm, n_units, gas_m3_s0, gas_m3_s1, electricity_kwh_s0, electricity_kwh_s1, 
            ROUND(gas_m3_s0 * 1.9 + electricity_kwh_s0 * 0.45) AS operational_kg_s0, 
            CASE 
                WHEN status IN ('transformation - adding units', 'transformation - function change', 
                                'renovation - pre2020', 'renovation - post2020') THEN sqm * 126
                WHEN status = 'Bouw gestart' THEN sqm * 325
                WHEN status = 'Pand gesloopt' THEN sqm * 77
                ELSE 0 
            END AS embodied_kg_s0, 
            
            ROUND(gas_m3_s1 * 1.9 + electricity_kwh_s1 * 0.45) AS operational_kg_s1, 
            CASE 
                WHEN status IN ('transformation - adding units', 'transformation - function change', 
                                'renovation - pre2020', 'renovation - post2020') THEN sqm * 200
                WHEN status = 'Bouw gestart' THEN sqm * 550
                WHEN status = 'Pand gesloopt' THEN sqm * 77
                ELSE 0 
            END AS embodied_kg_s1, 
            wk_code, wk_geom
        FROM energy_use_per_building 
    ), 
    stats_per_building AS (
        SELECT 
            CASE 
                WHEN status = 'Bouw gestart' THEN sqm ELSE 0 
            END AS construction, 
            CASE 
                WHEN status IN ('transformation - adding units', 'transformation - function change') THEN sqm ELSE 0
            END AS transformation, 
            CASE 
                WHEN status IN ('renovation - pre2020', 'renovation - post2020') THEN sqm ELSE 0 
            END AS renovation, 
            CASE 
                WHEN status = 'Pand gesloopt' THEN sqm ELSE 0 
            END AS demolition, *
        FROM emissions_per_building 
    )
        
    SELECT {year} AS year, '{municipality}' AS municipality, wk_code, wk_geom,
        SUM(n_units) AS n_units, SUM(construction) AS construction, SUM(transformation) AS transformation, 
        SUM(renovation) AS renovation, SUM(demolition) AS demolition, 
        SUM(operational_kg_s0) AS operational_kg_s0, SUM(operational_kg_s1) AS operational_kg_s1, 
        SUM(embodied_kg_s0) AS embodied_kg_s0, SUM(embodied_kg_s1) AS embodied_kg_s1
    FROM stats_per_building
    GROUP BY wk_code, wk_geom
    '''

    gdf = gpd.read_postgis(query, engine, geom_col='wk_geom')
    gdf_all = pd.concat([gdf_all, gdf])
    print(f'Year {year} complete')


Year 2012 complete
Year 2013 complete
Year 2014 complete
Year 2015 complete
Year 2016 complete
Year 2017 complete
Year 2018 complete
Year 2019 complete
Year 2020 complete
Year 2021 complete


In [22]:
gdf_all

embodied_s0 = gdf_all.embodied_kg_s0.sum()
embodied_s1 = gdf_all.embodied_kg_s1.sum()

(embodied_s1 - embodied_s0) / embodied_s0 * 100

58.730158730158735

In [37]:
test = gdf_all[gdf_all.wk_code == 'WK098312']

embodied_s0 = test.embodied_kg_s0.sum()
embodied_s1 = test.embodied_kg_s1.sum()
diff_pct = (embodied_s1 - embodied_s0) / embodied_s0 * 100

print(f'embodied s0: {embodied_s0}, embodied s1: {embodied_s1}, diff: {diff_pct}%')

embodied s0: 4232088.0, embodied s1: 6717600.0, diff: 58.730158730158735%


In [39]:
(200-126) / 126 * 100

58.730158730158735

In [24]:
gdf_all[gdf_all.construction > 0]

,year,municipality,wk_code,wk_geom,n_units,construction,transformation,renovation,demolition,operational_kg_s0,operational_kg_s1,embodied_kg_s0,embodied_kg_s1


In [42]:
query = ''' 
SELECT id_pand, status, sqm, n_units, geom, geom_28992, wk_code, municipality
FROM housing_nl
WHERE municipality = 'Delft'
	AND ahn_version IS NULL
	AND status = 'Bouw gestart'
    AND wk_code IS NOT NULL 
'''

df = pd.read_sql(query, engine)
df.head(2)

,id_pand,status,sqm,n_units,geom,geom_28992,wk_code,municipality
0,0503100000033023,Bouw gestart,215,1,0103000020E61000000100000005000000DA6B8933276D...,010300002040710000010000000500000025068195118E...,WK050326,Delft
1,0503100000033273,Bouw gestart,94,1,0103000020E61000000100000006000000A0F2A23A066C...,0103000020407100000100000006000000643BDF4F078D...,WK050311,Delft


In [56]:
query = f''' 
WITH emissions_with_municipality AS (
    SELECT b.municipality, a.wk_code, a.year, a.n_units, 
        a.construction, a.transformation, a.renovation, a.demolition,
		a.embodied_kg_s0, a.embodied_kg_s1, 
		a.operational_kg_s0, a.operational_kg_s1
	FROM emissions_all_wijk_s1 a 
	LEFT JOIN key_wijk2012_to_municipality2022 b 
	ON a.wk_code = b.wk_code
), 
emissions_municipality AS (
    SELECT municipality, 
        SUM(operational_kg_s0) AS operational_kg_s0, SUM(operational_kg_s1) AS operational_kg_s1, 
        SUM(embodied_kg_s0) AS embodied_kg_s0, SUM(embodied_kg_s1) AS embodied_kg_s1, 
        SUM(operational_kg_s0 + embodied_kg_s0) AS total_kg_s0, SUM(operational_kg_s1 + embodied_kg_s1) AS total_kg_s1,
        SUM(construction) AS construction, SUM(transformation) AS transformation,
        SUM(renovation) AS renovation, SUM(demolition) AS demolition
    FROM emissions_with_municipality
    GROUP BY municipality
), 
emissions_stats AS (
    SELECT municipality, 
        operational_kg_s0, operational_kg_s1, embodied_kg_s0, embodied_kg_s1, total_kg_s0, total_kg_s1,
        construction, transformation, renovation, demolition,
        operational_kg_s1 - operational_kg_s0 AS diff_operational,
        (operational_kg_s1 - operational_kg_s0) / operational_kg_s0 * 100 AS diff_operational_pct,
        embodied_kg_s1 - embodied_kg_s0 AS diff_embodied,
        (embodied_kg_s1 - embodied_kg_s0) / embodied_kg_s0 * 100 AS diff_embodied_pct,
        total_kg_s1 - total_kg_s0 AS diff_total,
        (total_kg_s1 - total_kg_s0) / total_kg_s0 * 100 AS diff_total_pct
    FROM emissions_municipality 
), 
municipalities AS (
    SELECT "GM_NAAM", geometry FROM nl_gemeenten WHERE "H2O" = 'NEE'
)
SELECT a.*, b.geometry
FROM emissions_stats a
LEFT JOIN municipalities b
ON a.municipality = b."GM_NAAM"
ORDER BY a.diff_total DESC
'''

gdf = gpd.read_postgis(query, engine, geom_col='geometry')

In [57]:
gdf

,municipality,operational_kg_s0,operational_kg_s1,embodied_kg_s0,embodied_kg_s1,total_kg_s0,total_kg_s1,construction,transformation,renovation,demolition,diff_operational,diff_operational_pct,diff_embodied,diff_embodied_pct,diff_total,diff_total_pct,geometry
0,Venlo,2.053989e+09,2.009470e+09,2.379921e+09,3.778678e+09,4.433910e+09,5.788148e+09,158411.0,18248529.0,171930.0,96883.0,-44519022.0,-2.167442,1.398756e+09,58.773220,1.354237e+09,30.542733,"POLYGON ((208343.558 383198.921, 208337.757 38..."
1,Aalsmeer,6.649024e+08,6.019152e+08,6.244790e+07,1.025007e+08,7.273503e+08,7.044159e+08,140145.0,23151.0,91987.0,31083.0,-62987223.0,-9.473153,4.005284e+07,64.138001,-2.293439e+07,-3.153142,"POLYGON ((114958.081 475498.727, 114953.183 47..."
2,Vlaardingen,1.336738e+09,1.194434e+09,1.302042e+08,2.109982e+08,1.466943e+09,1.405433e+09,264946.0,64567.0,221665.0,104305.0,-142304158.0,-10.645624,8.079402e+07,62.051791,-6.151014e+07,-4.193084,"POLYGON ((84553.223 437732.328, 84585.189 4376..."


# Fixing missing construction / demolition data 

I've done the following to fix the problem. 
- ✅ update s1EnergyEfficiency query so that sqm of construction, renovation, transformation, and demolition are shown in the final table emissions_all_wijk_s1 
- ✅ change queries so that we don't delete and re-make entire tables everytime we re-run the code. Instead, delete, re-make, and re-insert specific rows. Reference code to making housing_nl 
- ✅ re-run queries 

# Testing newly created results - solved 🎉

I've tested the results, and they look good! problem solved! 

In [63]:
query = f''' 
WITH emissions_with_municipality AS (
    SELECT b.municipality, a.wk_code, a.year, a.n_units, 
		a.embodied_kg_s0, a.embodied_kg_s1, 
		a.operational_kg_s0, a.operational_kg_s1
	FROM emissions_all_wijk_s1 a 
	LEFT JOIN key_wijk2012_to_municipality2022 b 
	ON a.wk_code = b.wk_code
), 
emissions_municipality AS (
    SELECT municipality, 
        SUM(operational_kg_s0) AS operational_kg_s0, SUM(operational_kg_s1) AS operational_kg_s1, 
        SUM(embodied_kg_s0) AS embodied_kg_s0, SUM(embodied_kg_s1) AS embodied_kg_s1, 
        SUM(operational_kg_s0 + embodied_kg_s0) AS total_kg_s0, SUM(operational_kg_s1 + embodied_kg_s1) AS total_kg_s1
    FROM emissions_with_municipality
    GROUP BY municipality
), 
emissions_stats AS (
    SELECT municipality, 
        operational_kg_s0, operational_kg_s1, embodied_kg_s0, embodied_kg_s1, total_kg_s0, total_kg_s1,
        operational_kg_s1 - operational_kg_s0 AS diff_operational,
        (operational_kg_s1 - operational_kg_s0) / operational_kg_s0 * 100 AS diff_operational_pct,
        embodied_kg_s1 - embodied_kg_s0 AS diff_embodied,
        (embodied_kg_s1 - embodied_kg_s0) / embodied_kg_s0 * 100 AS diff_embodied_pct,
        total_kg_s1 - total_kg_s0 AS diff_total,
        (total_kg_s1 - total_kg_s0) / total_kg_s0 * 100 AS diff_total_pct
    FROM emissions_municipality 
), 
municipalities AS (
    SELECT "GM_NAAM", geometry FROM nl_gemeenten WHERE "H2O" = 'NEE'
)
SELECT a.*, b.geometry
FROM emissions_stats a
LEFT JOIN municipalities b
ON a.municipality = b."GM_NAAM"
ORDER BY a.diff_total DESC
'''

gdf = gpd.read_postgis(query, engine, geom_col='geometry')

In [65]:
df = gdf[['municipality', 'embodied_kg_s0', 'embodied_kg_s1', 'diff_embodied_pct']].copy()
df['diff_embodied_pct_python'] = (df.embodied_kg_s1 - df.embodied_kg_s0) / df.embodied_kg_s0 * 100
df = df[(df.diff_embodied_pct == 58.730159)]
problem_municipalities = df.municipality.tolist()

print(f'{len(df) / len(gdf) * 100:.0f}% of the municipalities have the exact same result: 58.730159% increase in embodied emissions')
df.head()

0% of the municipalities have the exact same result: 58.730159% increase in embodied emissions


,municipality,embodied_kg_s0,embodied_kg_s1,diff_embodied_pct,diff_embodied_pct_python


In [66]:
fig = px.scatter(gdf, x='diff_operational_pct', y='diff_embodied_pct', 
                 hover_name='municipality', 
                 title="Percentage change in operational vs embodied emissions for Dutch municipalities")
fig.update_layout(width=800, height=700)
fig.show()

In [62]:
(550 - 325) / 325 * 100

69.23076923076923